### BoW. Синтетический пример

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
texts = ["Недоступен сайт ya.ru, не пингуется", 
         "не работает монитор", 
         "Не могу подключиться к серверу по ssh", 
         "на мониторе потемнения"]

In [3]:
cv = CountVectorizer(lowercase=False) 
tdata = cv.fit_transform(texts) 
ft = cv.get_feature_names() 

In [4]:
pd.DataFrame(tdata.todense(),
             columns=ft)

,ru,ssh,ya,Не,Недоступен,могу,монитор,мониторе,на,не,пингуется,по,подключиться,потемнения,работает,сайт,серверу
0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
2,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1
3,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0


Есть повторяющиеся токены ("Не" и "не"). Давайте исправим

In [5]:
cv = CountVectorizer(lowercase=True)

In [6]:
tdata = cv.fit_transform(texts) 
ft = cv.get_feature_names() 

In [7]:
pd.DataFrame(tdata.todense(),
             columns=ft)

,ru,ssh,ya,могу,монитор,мониторе,на,не,недоступен,пингуется,по,подключиться,потемнения,работает,сайт,серверу
0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0
1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
2,0,1,0,1,0,0,0,1,0,0,1,1,0,0,0,1
3,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0


Матрица стала немного меньше, но все равно есть что улучшить. 

Кажется разумным предположение что не все слова одинаково важны для понимания текста.

И отсюда есть идея придать словам некоторый вес. Допустим что есть слова, которые встречаются очень часто и поэтому не несут много информации, в то время как какие-то специализированные термины (например) встречаются реже, но при этом в конкретной задаче они куда как более информативны. 

Отсюда мы приходим к tfidf.

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(tdata)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': ft, 'weight': weights})

In [10]:
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
7,не,0.240088
4,монитор,0.161126
13,работает,0.161126
5,мониторе,0.144338
6,на,0.144338
12,потемнения,0.144338
0,ru,0.107509
1,ssh,0.107509
2,ya,0.107509
3,могу,0.107509


Видим проблему, что частица "не" имеет самый большой вес. Это непорядок и давайте это исправим.

In [11]:
cv = CountVectorizer(stop_words=[u"не", u"на", u"по"]) 
tdata = cv.fit_transform(texts) 
ft = cv.get_feature_names() 

In [12]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(tdata)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': ft, 'weight': weights})

Посмотрим как теперь выглядит наша матрица

In [13]:
pd.DataFrame(tdata.todense(),
             columns=ft)

,ru,ssh,ya,могу,монитор,мониторе,недоступен,пингуется,подключиться,потемнения,работает,сайт,серверу
0,1,0,1,0,0,0,1,1,0,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,1,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,1
3,0,0,0,0,0,1,0,0,0,1,0,0,0


In [14]:
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
4,монитор,0.176777
5,мониторе,0.176777
9,потемнения,0.176777
10,работает,0.176777
1,ssh,0.125000
3,могу,0.125000
8,подключиться,0.125000
12,серверу,0.125000
0,ru,0.111803
2,ya,0.111803


У нас есть повторяющиеся слова, которые обозначают одно и то же и просто являются словоформами.
Давайте это исправим.

In [15]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

Пройдемся в цикле (так делать нехорошо, но для наших целей сойдет)

In [29]:
texts_morph = []
for text in texts:
    texts_morph.append(" ".join([morph.parse(word)[0].normal_form for word in text.split()]))

In [30]:
cv = CountVectorizer(stop_words=[u"не", u"на", u"по"]) 
tdata = cv.fit_transform(texts_morph) 
ft = cv.get_feature_names() 

In [31]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(tdata)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': ft, 'weight': weights})

In [32]:
pd.DataFrame(tdata.todense(),
             columns=ft)

,ru,ssh,ya,монитор,мочь,недоступный,пинговаться,подключиться,потемнение,работать,сайт,сервер
0,1,0,1,0,0,1,1,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,1,0,0
2,0,1,0,0,1,0,0,1,0,0,0,1
3,0,0,0,1,0,0,0,0,1,0,0,0


In [33]:
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
3,монитор,0.309565
8,потемнение,0.196322
9,работать,0.196322
1,ssh,0.125000
4,мочь,0.125000
7,подключиться,0.125000
11,сервер,0.125000
0,ru,0.111803
2,ya,0.111803
5,недоступный,0.111803
